In [1]:
import warnings
warnings.filterwarnings("ignore")
from model import *
import numpy as np
import cv2 # lib image processing 

Using TensorFlow backend.


In [2]:
model=graph()


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


read image and mask

In [3]:
Image = cv2.imread("dataset/X_train.png",cv2.IMREAD_UNCHANGED)
Image = np.expand_dims(Image,axis=2)
print("image shape {}".format(Image.shape))

Mask = cv2.imread("dataset/Y_train.png",cv2.IMREAD_UNCHANGED)
Mask = np.expand_dims(Mask,axis=2)
print("mask shape {}".format(Mask.shape))

X = crop_from_photo(Image,shift=25,size=100)/255
Y = crop_from_photo(Mask,shift=25,size=100)/255

print("Image pieces shape {}".format(X.shape))
print("Mask pieces shape {}".format(Y.shape))

N=X.shape[0]
N = N*6//100
x_train = X[:N,:,:,:]
y_train = Y[:N,:,:,:]
x_val = X[N:,:,:,:]
y_val=Y[N:,:,:,:]

print("")


image shape (960, 1280, 1)
mask shape (960, 1280, 1)
Image pieces shape (2028, 100, 100, 1)
Mask pieces shape (2028, 100, 100, 1)



0.65882355

training neural network 10 iterations (epochs).

In [4]:
model.fit_generator(generator=augmentation(x_train,y_train),
                   validation_data=(x_val,y_val),
                   steps_per_epoch=50,
                   epochs=10)


Epoch 1/10
50/50 [==============================] - 5s 105ms/step - loss: 0.0328 - val_loss: 0.0153
Epoch 2/10
50/50 [==============================] - 2s 49ms/step - loss: 0.0120 - val_loss: 0.0111
Epoch 3/10
50/50 [==============================] - 2s 48ms/step - loss: 0.0107 - val_loss: 0.0104
Epoch 4/10
50/50 [==============================] - 2s 48ms/step - loss: 0.0099 - val_loss: 0.0092
Epoch 5/10
50/50 [==============================] - 2s 49ms/step - loss: 0.0093 - val_loss: 0.0092
Epoch 6/10
50/50 [==============================] - 2s 49ms/step - loss: 0.0091 - val_loss: 0.0092
Epoch 7/10
50/50 [==============================] - 2s 49ms/step - loss: 0.0087 - val_loss: 0.0087
Epoch 8/10
50/50 [==============================] - 2s 49ms/step - loss: 0.0085 - val_loss: 0.0085
Epoch 9/10
50/50 [==============================] - 2s 48ms/step - loss: 0.0082 - val_loss: 0.0084
Epoch 10/10
50/50 [==============================] - 2s 49ms/step - loss: 0.0078 - val_loss: 0.0085


In [53]:
from keras.models import load_model

model.save('model.h5')

In [37]:
def recognize(image):
    shift = 100
    size = 100
    image = np.expand_dims(image,axis=2)/255
    output = np.zeros([image.shape[0],image.shape[1]],dtype="float32")
    buf1 = np.zeros([1,size,size,1],dtype="float32")
    buf2 = np.zeros([size,size],dtype="float32")
    buf3=np.zeros([size,size],dtype="float32")
    for x_shift in range(math.ceil(image.shape[0]/shift)):
        for y_shift in range(math.ceil(image.shape[1]/shift)):
            x = shift*x_shift
            y = shift*y_shift
            piece = image[x:x+size,y:y+size,:]
            buf1[0,:piece.shape[0],:piece.shape[1],:]=piece
            buf2 = (model.predict(buf1))[0,:,:,0]
            output[x:x+piece.shape[0],y:y+piece.shape[1]]+=buf2[:piece.shape[0],:piece.shape[1]]
    return output

In [38]:
import os
import csv
from matplotlib import pyplot as plt

In [54]:
txt_files=os.listdir("Coli/")
frames_files=os.listdir("frames_gray/")
txt_files.sort()
frames_files.sort()

In [57]:
for i in range(len(txt_files)):
    with open("Coli/"+txt_files[i],"r") as f:
        reader = csv.reader(f,delimiter=':')
        bacteria_coordinates=[]
        for row in reader:
            bacteria_coordinates.append([float(row[1]),float(row[0])])
    frame=cv2.imread("frames_gray/"+frames_files[i],cv2.IMREAD_UNCHANGED) #load frame
       
    bac_coord = np.array(bacteria_coordinates)# load coord to matrix
    bac_coord= bac_coord.astype("int32") # convert to integer
    
    bac_rec = recognize(frame)
    print(bac_rec.max())    
    frame=cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    print(frame.shape)
    frame[:,:,0]=frame[:,:,0]+bac_rec*200
    for k in range(bac_coord.shape[0]):
        cv2.circle(frame,center=(bac_coord[k,0],bac_coord[k,1]),radius=3,color=(0,0,255),thickness=-1)
    
    cv2.imwrite("result/%s.png"%(i),frame)
    

0.46126837
(960, 1280, 3)
0.45015514
(960, 1280, 3)
0.45079327
(960, 1280, 3)
0.45050848
(960, 1280, 3)
0.44923714
(960, 1280, 3)
0.4509689
(960, 1280, 3)
0.4589247
(960, 1280, 3)
0.4491946
(960, 1280, 3)
0.45238778
(960, 1280, 3)
0.4505205
(960, 1280, 3)
0.44961682
(960, 1280, 3)
0.4536742
(960, 1280, 3)
0.45081133
(960, 1280, 3)
0.45709616
(960, 1280, 3)
0.4480876
(960, 1280, 3)
0.44957238
(960, 1280, 3)
0.4482057
(960, 1280, 3)
0.4471716
(960, 1280, 3)
0.4517864
(960, 1280, 3)
0.4492828
(960, 1280, 3)
0.4536371
(960, 1280, 3)
0.44967067
(960, 1280, 3)
0.4492913
(960, 1280, 3)
0.4499035
(960, 1280, 3)
0.44721332
(960, 1280, 3)
0.44750002
(960, 1280, 3)
0.44919422
(960, 1280, 3)
0.44985077
(960, 1280, 3)
0.4476726
(960, 1280, 3)
0.44886398
(960, 1280, 3)
0.4509699
(960, 1280, 3)
0.4506657
(960, 1280, 3)
0.44858134
(960, 1280, 3)
0.4531035
(960, 1280, 3)
0.45255443
(960, 1280, 3)
0.44789004
(960, 1280, 3)
0.44777656
(960, 1280, 3)
0.45395502
(960, 1280, 3)
0.45042378
(960, 1280, 3)
0.4

IndexError: list index out of range